# Example using Backtesting.py

Docs: https://kernc.github.io/backtesting.py

![](../images/screenshot.png)

In [44]:
import datetime
import warnings

import pandas as pd
import talib
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

warnings.filterwarnings("ignore")  # Ignore warnings to clean up the outputs.

## Get finance data.

Backtesting requires data to have columns Open, Close, High, Low.

In [45]:
# Get finance data.

ticker: str = "GOOG"
ticker_df: yf.Ticker = yf.Ticker(ticker)

df: pd.DataFrame = ticker_df.history(period="1y")

## How to use Backtesting.py

Create a child of `Strategy` class and define `init` and `next`.

You can use other Python libraries like TA-Lib or pandas-ta to include commonly
used strategies.

In [46]:
# You will have to inherit and define `Strategy` class.
print(help(Strategy))

Help on class Strategy in module backtesting.backtesting:

class Strategy(builtins.object)
 |  Strategy(broker, data, params)
 |  
 |  A trading strategy base class. Extend this class and
 |  override methods
 |  `backtesting.backtesting.Strategy.init` and
 |  `backtesting.backtesting.Strategy.next` to define
 |  your own strategy.
 |  
 |  Methods defined here:
 |  
 |  I(self, func: 'Callable', *args, name=None, plot=True, overlay=None, color=None, scatter=False, **kwargs) -> 'np.ndarray'
 |      Declare an indicator. An indicator is just an array of values
 |      (or a tuple of such arrays in case of, e.g., MACD indicator),
 |      but one that is revealed gradually in
 |      `backtesting.backtesting.Strategy.next` much like
 |      `backtesting.backtesting.Strategy.data` is.
 |      Returns `np.ndarray` of indicator values.
 |      
 |      `func` is a function that returns the indicator array(s) of
 |      same length as `backtesting.backtesting.Strategy.data`.
 |      
 |      

In [47]:
# Example using RSI cross.


class Rsi(Strategy):  # Inherit from `Strategy` class.
    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    starting_equity: float = 10000
    per_trade_share_quantity: float = 2.0

    def init(self):
        """Run once.

        Backtests.py is unusual in the way it handles series data for the
        `next()` function.
        """
        # Using I is indicator; an array of values which is iterated through
        # with `next()`
        # talib.RSI calculates RSI indicator.
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        print(f"Starting cash: ${self.starting_equity:,.2f}")

    def next(self):
        """Define strategy.

        Reads one at a time.

        self.data.Close is a Series. Calling next() will take the last value of
        the Series.

        Series and arrays defined in the class will be iterated in `next()`
        function. So if you defined `self.list = pd.Series()` in init(), then
        you would iterate in `next()` function by using `self.list`.
        """

        # print(f"NEXT {self.data} {self.position} {self.upper_bound} {self.lower_bound}")

        if crossover(self.rsi, self.upper_bound):
            # Specify percent of position to sell `size=`
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            # If size not specified, then uses all equity
            # Set stop loss, limits in buy params
            self.buy()

In [48]:
# Execute backtest.
#
# You will be able to see your output print statements in the terminal.
# Run will also show you the backtest results.

backtest: Backtest = Backtest(
    data=df, strategy=Rsi, cash=Rsi.starting_equity, commission=0.02
)
backtest.run()

Starting cash: $10,000.00


Start                     2024-06-10 00:00...
End                       2025-06-09 00:00...
Duration                    364 days 00:00:00
Exposure Time [%]                        24.8
Equity Final [$]                  10702.18945
Equity Peak [$]                   10741.12905
Commissions [$]                     407.83399
Return [%]                            7.02189
Buy & Hold Return [%]                -3.25613
Return (Ann.) [%]                     7.08001
Volatility (Ann.) [%]                25.48156
CAGR [%]                              4.81034
Sharpe Ratio                          0.27785
Sortino Ratio                         0.42783
Calmar Ratio                          0.39763
Alpha [%]                             8.79914
Beta                                  0.54581
Max. Drawdown [%]                   -17.80538
Avg. Drawdown [%]                    -6.91112
Max. Drawdown Duration      222 days 00:00:00
Avg. Drawdown Duration       62 days 00:00:00
# Trades                          

In [49]:
# Create backtest charts.
#
# Must run `backtest.run()` before running this.

backtest.plot()

GridPlot(id='p3324', ...)

In [50]:
# Save chart in `./charts` folder and show in browser.
backtest.plot(
    filename=f"./charts/{ticker}_{backtest._strategy.__name__}_{datetime.datetime.now().strftime('%Y_%m_%d-%H_%M_%S')}"
)

GridPlot(id='p3729', ...)